In [1]:
!pip install fastapi uvicorn ultralytics torch pillow opencv-python numpy pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
from pyngrok import ngrok

# Authenticate ngrok session (REQUIRED for persistent tunnels)
ngrok.set_auth_token("2tATxrcH0i1eLwLI83F76BilqGo_ZEsQwU8e6EbFkj7jokL1")  # Replace with your actual token

# Start tunnel
ngrok_tunnel = ngrok.connect(8000)
print(f'Public URL: {ngrok_tunnel.public_url}')


Public URL: https://7e1a-35-190-183-127.ngrok-free.app


In [ ]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from ultralytics import YOLO
from PIL import Image
import io
import numpy as np
import base64
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Apply nest_asyncio (Needed for running FastAPI in notebooks)
nest_asyncio.apply()

# Load YOLO model
model = YOLO("/content/drive/MyDrive/final_project/best.pt")

# Initialize FastAPI app
app = FastAPI()

# ✅ Root Endpoint
@app.get("/")
def home():
    return {"message": "FastAPI is running! Use /predict/ to send base64 images."}

# ✅ Handle favicon requests
@app.get("/favicon.ico")
def favicon():
    return JSONResponse(content="")

# ✅ Request model
class ImageRequest(BaseModel):
    image_base64: str

# ✅ Prediction Endpoint
@app.post("/predict/")
async def predict(request: ImageRequest):
    try:
        # Decode base64 image
        image_data = base64.b64decode(request.image_base64)
        img = Image.open(io.BytesIO(image_data)).convert("RGB")
        img_np = np.array(img)

        # Run YOLO model prediction
        results = model.predict(source=img_np, conf=0.25)

        # Extract detected labels
        labels = []
        for result in results:
            for box in result.boxes:
                class_id = int(box.cls[0])  # Get class ID
                label = model.names[class_id]  # Get class name from model
                labels.append(label)

        return {"detected_labels": labels}
    except Exception as e:
        return {"error": str(e)}

# ✅ Authenticate ngrok
ngrok.set_auth_token("2tATxrcH0i1eLwLI83F76BilqGo_ZEsQwU8e6EbFkj7jokL1")  # Replace with your actual token

# ✅ Start ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print(f'Public URL: {ngrok_tunnel.public_url}')

# ✅ Start FastAPI server
uvicorn.run(app, host='0.0.0.0', port=8000)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Public URL: https://670b-35-190-183-127.ngrok-free.app


INFO:     Started server process [743]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



0: 448x640 1 Spiral, 1269.7ms
Speed: 34.9ms preprocess, 1269.7ms inference, 34.1ms postprocess per image at shape (1, 3, 448, 640)
INFO:     202.47.48.164:0 - "POST /predict/ HTTP/1.1" 200 OK

0: 448x640 1 Spiral, 1015.3ms
Speed: 4.4ms preprocess, 1015.3ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
INFO:     202.47.48.164:0 - "POST /predict/ HTTP/1.1" 200 OK

0: 448x640 1 Spiral, 1595.1ms
Speed: 5.4ms preprocess, 1595.1ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
INFO:     202.47.48.164:0 - "POST /predict/ HTTP/1.1" 200 OK

0: 448x640 1 Spiral, 1135.1ms
Speed: 12.7ms preprocess, 1135.1ms inference, 2.4ms postprocess per image at shape (1, 3, 448, 640)
INFO:     202.47.48.164:0 - "POST /predict/ HTTP/1.1" 200 OK

0: 384x640 (no detections), 1671.4ms
Speed: 5.6ms preprocess, 1671.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
INFO:     202.47.48.164:0 - "POST /predict/ HTTP/1.1" 200 OK

0: 448x640 1 Spiral, 1120.1ms
Spee